In [1]:
import csv
import os, random
import glob
from pydub import AudioSegment
import xlsxwriter

os.chdir(path=r"D:\02_batch_2")
print("Current Working Directory " , os.getcwd())

Current Working Directory  D:\02_batch_2


C:\Users\koool\AppData\Local\Programs\Python\Python39\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
# Step 1: Open dir and Select random text file
def select_random_file():
    os.chdir(dataset_path)
    myFiles = glob.glob('*.txt')
    text_file = random.choice(myFiles) #change dir name to whatever
    audio_file = text_file.replace(".txt", ".wav")
    read_random_line(text_file, audio_file)

# Step 2: Read random line
def read_random_line(text_file, audio_file):
    global total_duration
    global current_duration    

    try:
        lines = open(dataset_path + text_file, encoding="utf8").read().splitlines()
        random_line = random.choice(lines)

        columns = random_line.split("\t")
        start = float(columns[2])
        end = float(columns[3])
        transcript = columns[4]
        duration = end - start

        segment_name = audio_file.replace(".wav", "") + '_' + str(start)  + '_' + str(end) + ".wav"
     #   print(columns)

     #   print(segment_name, duration)
        # if segment is not valid read another one
        if segment_name in files and duration < 1:
            read_random_line(text_file, audio_file)
        else:
            total_duration += duration 
            current_duration += duration
            split_audio(audio_file, start, end,transcript, segment_name)
    except ValueError:
        read_random_line(text_file, audio_file)


# Step 3: Split the audio into segment 
def split_audio(audio_file, start, end, transcript, segment_name):
    t1 = start * 1000 #Works in milliseconds
    t2 = end * 1000
    new_audio_path = current_path+segment_name
    new_audio = AudioSegment.from_wav(dataset_path + audio_file)
    new_audio = new_audio[t1:t2]
    new_audio.export(new_audio_path, format="wav") #Exports to a wav file in the current path.
    write_segment(new_audio_path, transcript)
    write_segment_excel(segment_name,current_path,audio_file,start,end,transcript)

# Step 4: Write the new segment and write in the CSV
def write_segment(segment_name, transcript):
    writer.writerow([segment_name, transcript])
    files.append(segment_name)

# Step 5: Write the new segment and write in the excel
def write_segment_excel(segment_name,current_path,audio_file,start,end,transcript):
    r_start = int(start)
    r_end = int(end)
    segment_name = audio_file.replace(".wav", "") + '_' + str(r_start)  + '_' + str(r_end) + ".wav"
    audio_file = dataset_path + "/" + audio_file
    writer_excel.writerow([segment_name,current_path,audio_file,start,end,transcript])
    #print(segment_name," ",current_path," ",audio_file," ",start," ",end," ",transcript)

In [3]:
for f in sorted(os.listdir(os.getcwd())):
     
    ######################[CAN BE UPDATED]###################
    dialect = f
    # The root folder where the dialect is exist
    dataset_path = r"D:/02_batch_2/{}/".format(dialect)
    wanted_duration = 0.84 #in hours
    number_of_dir = 10 #total number of files you want to separate the segments to 
    ########################################################

    output_path = r'D:/batch_sample_2/{}_segments/'.format(dialect)

    ######################[DO NOT CHANGE]###################
    files = []
    segments_path = output_path
    dir_counter = 0
    for root, dirs, files in os.walk(segments_path, topdown=False):
        dir_counter = len(dirs) + 1
        break

    current_path = segments_path +"/"+ str(dir_counter) + "/"
    total_duration = 0
    current_duration = 0

    if not os.path.exists(segments_path):
        os.mkdir(segments_path)

    if not os.path.exists(current_path):
        os.mkdir(current_path)

    # Create CSV
    header = ['segment path', 'transcript']
    f = open(segments_path + str(dialect) + "_segments.csv", "w", encoding="utf8")
    writer = csv.writer(f)
    writer.writerow(header)

    # Create excel
    header_excel = ['segment_name','current_path','audio_file','start','end','transcript']
    f_excel = open(segments_path + str(dialect) + "_details.csv", "w", encoding="utf8")
    writer_excel = csv.writer(f_excel)
    writer_excel.writerow(header_excel)

    ########################################################


    if __name__ == "__main__":
        while total_duration < wanted_duration * 60 * 60:
            if current_duration > wanted_duration* 60 * 60 / number_of_dir:
                dir_counter += 1
                current_path = segments_path +"/"+ str(dir_counter) + "/"
                current_duration = 0

                if not os.path.exists(current_path):
                    os.mkdir(current_path)
            
            print(total_duration)
            select_random_file()



0
2.824926282979959
3.324893872909459
7.549118816980459
8.748822032151459
9.32372352308547
16.323159600143477
19.447705400383477
22.97209899835348
23.77208232589768
25.77206886468268
27.82166893195478
29.69650161563479
31.296306601783787
32.42106991312379
35.92022164248378
36.969924910073786
38.54483891163379
39.8697541615918
43.49443681181181
49.3443143756418
49.944265689801796
50.769167216293795
57.51797888466379
61.717331381853796
62.31722575237379
64.64191977970579
73.8150145789558
74.214976185422
76.214812072952
77.639529231482
81.413745347793
83.76347632422295
93.18750962770295
96.01247076047096
98.48716445741101
104.21160956102102
106.11139786981101
107.88609377660701
108.511058365837
110.96086514436699
112.810794634597
137.507204639667
142.181802827257
145.256511358063
145.85645883120299
146.73124987246499
149.88121268237498
151.60590410761398
154.90566500300395
155.43060674734394
157.23044916676295
159.75533828062294
160.08028260158292
162.40484647990291
177.5526111950229
180.